In [1]:
import os 
import shutil
from pupil_apriltags import Detector
import cv2
import numpy as np
import json
import scipy as sp
import socket
import pickle
from IPython.display import clear_output
# !pip install scipy

In [2]:
#load calibration file
def load_calibration_file(calibration_file_path):
    #reconstruct the camera matrix and distortion coefficients
    with open(calibration_file_path, 'r') as infile:
        reconstruction = json.load(infile)
        mtx = np.array(reconstruction['mtx'])
        dist = np.array(reconstruction['dist'])
    return mtx, dist
calibration_path = r'calibration_data.json'
mtx, dist = load_calibration_file(calibration_path)
fx, fy, cx, cy = mtx[0,0], mtx[1,1], mtx[0,2], mtx[1,2]
mtx = [fx, fy, cx, cy]

In [120]:
class PoseData:
    tag_id: int
    #quaternion
    w: float
    x: float
    y: float
    z: float
    #translation
    tx: float
    ty: float
    tz: float
    def __init__(self, tag_id, w, x, y, z, tx, ty, tz):
        self.tag_id = tag_id
        self.w = w
        self.x = x
        self.y = y
        self.z = z
        self.tx = tx
        self.ty = ty
        self.tz = tz
def connect_to_udp_server(ip, port):
    sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    server_address = (ip, port)
    print('connecting on {} port {}'.format(*server_address))
    sock.connect(server_address)
    return sock

def send_data_to_udp_server(sock, data):
    sock.sendall(pickle.dumps(data))
    
s = connect_to_udp_server('localhost', 1334)

connecting on localhost port 1334


In [121]:
def detector_superimpose(img, detector, tag_size=0.16):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    detections = detector.detect(gray, estimate_tag_pose= True, camera_params=(mtx), tag_size=tag_size)
    for d in detections:
        cv2.circle(img, (int(d.center[0]), int(d.center[1])), 5, (0, 0, 255), -1)
        cv2.putText(img, str(d.tag_id), (int(d.center[0]), int(d.center[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        #draw corners
        pose_data = d.pose_R, d.pose_t
        rvec, tvec = pose_data[0], pose_data[1]
        #rotation matrix to quaternion
        quat = sp.spatial.transform.Rotation.from_matrix(rvec).as_quat()
        euler = sp.spatial.transform.Rotation.from_matrix(rvec).as_euler('xyz', degrees=True)
        #draw euler angles
        str_euler = "x: " + str(round(euler[0], 2)) + " y: " + str(round(euler[1], 2)) + " z: " + str(round(euler[2], 2))
        cv2.putText(img, str_euler, (int(d.center[0]), int(d.center[1]+20)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        corner_id = 0
        for c in d.corners:
            cv2.circle(img, (int(c[0]), int(c[1])), 5, (0, 255, 0), -1)
            #draw number
            cv2.putText(img, str(corner_id), (int(c[0]), int(c[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            corner_id += 1
        #send data to udp server
        pose_data = PoseData(d.tag_id, quat[0], quat[1], quat[2], quat[3], tvec[0], tvec[1], tvec[2])
        send_data_to_udp_server(s, data=pose_data)
    return img

class tag:
    def __init__(self, center, id):
        self.center = center
        self.id = id
        self.tags = []
    def appendTag(self, tag):
        self.tags.append(tag)
#why does tags act like a static variable?
#because it is a static variable

'''
structur of the skeleton so far
    left elbow   right elbow id = 8, 9
    left shoulder right shoulder id = 6, 7
            chest(root) id = 0
            hip id = 1
    left_thigh    right_thigh id = 2, 3
    left_shin   right_shin  id = 4, 5
'''
left_elbow = tag(center=(0,0), id=8)
right_elbow = tag(center=(0,0), id=9)
left_shoulder = tag(center=(0,0), id=6)
right_shoulder = tag(center=(0,0), id=7)
chest = tag(center=(0,0), id=0)
hip = tag(center=(0,0), id=1)
left_thigh = tag(center=(0,0), id=2)
right_thigh = tag(center=(0,0), id=3)
left_shin = tag(center=(0,0), id=4)
right_shin = tag(center=(0,0), id=5)
#join the tags
chest.appendTag(hip)
chest.appendTag(left_shoulder)
chest.appendTag(right_shoulder)
left_shoulder.appendTag(left_elbow)
right_shoulder.appendTag(right_elbow)
hip.appendTag(left_thigh) 
hip.appendTag(right_thigh)
left_thigh.appendTag(left_shin)
right_thigh.appendTag(right_shin)

tag_list = [chest, hip, left_thigh, right_thigh, left_shin, right_shin, left_shoulder, right_shoulder, left_elbow, right_elbow]

def rand_color():
    return np.random.randint(0, 255, 3)

def detector_superimpose_test(img, detector, tag_size=0.16):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    detections = detector.detect(gray, estimate_tag_pose= True, camera_params=(mtx), tag_size=tag_size)
    
    for d in detections:
        
        cv2.circle(img, (int(d.center[0]), int(d.center[1])), 5, (0, 0, 255), -1)
        cv2.putText(img, str(d.tag_id), (int(d.center[0]), int(d.center[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        #draw corners
        corner_id = 0
        for c in d.corners:
            cv2.circle(img, (int(c[0]), int(c[1])), 5, (0, 255, 0), -1)
            #draw number
            cv2.putText(img, str(corner_id), (int(c[0]), int(c[1])), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            corner_id += 1
        for t in tag_list:
            if d.tag_id == t.id:
                t.center = [d.center[0], d.center[1]]
    #draw skeleton
    for t in tag_list:
        for t2 in t.tags:
            if t2.center != (0,0):
                color = rand_color()
                color = (int(color[0]), int(color[1]), int(color[2]))
                cv2.line(img, [int(t.center[0]), int(t.center[1])], [int(t2.center[0]), int(t2.center[1])], color, 2)
    return img
    
def live_feed(camera_id):
    cap = cv2.VideoCapture(camera_id)
    detector = Detector(families='tag36h11', nthreads=1, quad_decimate=1.0, quad_sigma=0.0, refine_edges=1, decode_sharpening=0.25, debug=0)
    while True:
        ret, frame = cap.read()
        if ret:
            frame = detector_superimpose(frame, detector)
            cv2.imshow('frame', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            print("Camera is not working")
            break
    cap.release()
    cv2.destroyAllWindows()

live_feed(1)


In [122]:
for t in tag_list:
    print(str(t.id) + ' connected to: ', end='')
    for t2 in t.tags:
        print(str(t2.id) + ' ', end='')
    print()

0 connected to: 1 6 7 
1 connected to: 2 3 
2 connected to: 4 
3 connected to: 5 
4 connected to: 
5 connected to: 
6 connected to: 8 
7 connected to: 9 
8 connected to: 
9 connected to: 
